<a href="https://colab.research.google.com/github/EnPaiva93/tume/blob/colab/metabook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon Latin America 2024

## Prompts

In [107]:
generation_prompt = """Usted es un escritor de podcast educativo de clase mundial, especializado en crear conversaciones atractivas y educativas sobre diversos temas. Su tarea es generar un diálogo de podcast entre dos oradores sobre el siguiente tema:


<tema>

{topic}

</tema>


Antes de comenzar el diálogo, planifique la estructura de la conversación dentro de las etiquetas <planificacion_podcast>. En esta sección:


1. Divida el tema en 3-5 subtemas principales para cubrir durante la conversación.

2. Para cada subtema, anote:

   - Una pregunta clave que el Orador 2 podría hacer.

   - Un punto principal que el Orador 1 debería explicar.

   - Una posible anécdota o analogía que el Orador 1 podría usar para ilustrar el punto.

   - 1-2 potenciales tangentes interesantes o preguntas inusuales que el Orador 2 podría plantear.

3. Planifique una introducción atractiva y una conclusión que resuma los puntos clave.

4. Sugiera 2-3 lugares donde el Orador 2 podría expresar confusión o pedir aclaraciones.

5. Planifique transiciones específicas entre cada subtema para mantener un flujo natural de la conversación.


Es aceptable que esta sección sea bastante larga para asegurar una planificación detallada.


Después de planificar la estructura, genere el diálogo siguiendo estas instrucciones:


1. Formato del diálogo:

   - Use "Orador 1:" y "Orador 2:" para indicar quién está hablando.

   - Incluya interrupciones naturales, como "ahh", "esss", "correcto", etc.

   - No incluya títulos de episodios o capítulos separados.


2. Roles de los oradores:

   - Orador 1: Experto que dirige la conversación. Debe ser un profesor cautivador que ofrece explicaciones claras, anécdotas interesantes y analogías efectivas.

   - Orador 2: Aprendiz curioso que mantiene el hilo de la conversación haciendo preguntas de seguimiento. Debe mostrar entusiasmo y ocasionalmente confusión, haciendo preguntas de confirmación interesantes.


3. Estructura de la conversación:

   - Comience con una bienvenida atractiva y un resumen del tema por parte del Orador 1.

   - El Orador 2 debe hacer preguntas relevantes y ocasionalmente ir por tangentes interesantes o inusuales.

   - El Orador 1 debe responder con explicaciones claras, utilizando anécdotas y analogías para ilustrar los puntos.

   - Mantenga un equilibrio entre la información educativa y el entretenimiento.


4. Contenido:

   - Asegúrese de que la conversación se mantenga centrada en el tema principal, aunque se permiten breves desvíos.

   - Incluya datos precisos y actualizados sobre el tema.

   - Use ejemplos del mundo real para hacer el contenido más relatable y comprensible.


5. Tono:

   - Mantenga un tono educativo pero accesible y amigable.

   - La conversación debe ser informativa pero también entretenida y atractiva para el oyente.

   - Los oradores son mujeres.


Presente su respuesta en el siguiente formato:


<resultado>

[Escriba aquí el resultado del tema de forma estructurada como wikipedia]

</resultado>


<dialogo>

[Inserte aquí el diálogo completo del podcast, comenzando directamente con el Orador 1 dando la bienvenida a los oyentes]

</dialogo>


Asegúrese de que el diálogo refleje la estructura planificada y cumpla con todas las instrucciones proporcionadas."""

## General

In [56]:
import nest_asyncio
nest_asyncio.apply()

In [57]:
%%capture
!pip install groq streamlit ffmpeg-python gTTS pymupdf4llm pydub TTS

In [58]:
# %%capture
# !pip install TTS

In [ ]:
# %%capture
# !pip install llama_index llama-index-embeddings-huggingface

In [ ]:
# %%capture
# !pip install llama-index-vector-stores-chroma llama-index-llms-groq

In [ ]:
# %%capture
# !pip install llama-index-llms-sambanovacloud
# !pip install sseclient-py

In [59]:
%%capture
!npm install localtunnel

In [ ]:
# %%capture
# %pip install llama-index-embeddings-openvino

In [ ]:
# %%capture
# %pip install llama-index-retrievers-bm25

In [4]:
from google.colab import userdata

In [5]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
SAMBANOVA_API_KEY = userdata.get('SAMBANOVA_API_KEY')

In [ ]:
# from llama_index.embeddings.huggingface_openvino import OpenVINOEmbedding

# OpenVINOEmbedding.create_and_save_openvino_model(
#     "sentence-transformers/all-MiniLM-L6-v2", "./models"
# )

In [ ]:
# import os
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.llms.groq import Groq
# from sentence_transformers import SentenceTransformer
# from llama_index.core import Settings

# # model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device = "cpu", backend="openvino")
# # model.save_pretrained("path/to/my/model")

# Settings.embed_model = OpenVINOEmbedding(model_id_or_path="./models", device="cpu")

# if not os.getenv("SAMBANOVA_API_KEY"):
#     os.environ["SAMBANOVA_API_KEY"] = SAMBANOVA_API_KEY

# # Settings.llm = SambaNovaCloud(
# #     model="Meta-Llama-3.1-70B-Instruct",
# #     max_tokens=1024,
# #     temperature=0.7,
# #     top_k=1,
# #     top_p=0.01,
# # )

# Settings.llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

## Utils

In [64]:
import io

def bytesio_to_ogg(input_bytesio: io.BytesIO, input_format: str = "wav", output_file: str = "temp.ogg"):
    """
    Convert audio data from a BytesIO object to an OGG file using ffmpeg-python.

    Args:
        input_bytesio (BytesIO): The input audio data as a BytesIO object.
        output_file (str): The path to save the converted OGG file.
    """
    # Reset the BytesIO pointer to the start
    input_bytesio.seek(0)

    # Use ffmpeg to read from BytesIO and write to an OGG file
    process = (
        ffmpeg
        .input('pipe:0', format=input_format)  # Adjust format if input is not WAV
        .output(
            output_file,
            format='ogg',
            acodec='libopus',  # Correct codec for OGG
            ac=1,              # Mono audio
            b='12k',           # Correct option for bitrate
            application='voip' # Libopus-specific option
        )
        .overwrite_output()
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    # Send BytesIO content to FFmpeg and capture any errors
    stdout, stderr = process.communicate(input=input_bytesio.read())

    if process.returncode != 0:
        raise RuntimeError(f"ffmpeg failed: {stderr.decode('utf-8')}")

    print(f"Converted to OGG: {output_file}")

# Función para extraer contenido entre etiquetas específicas
def extraer_contenido(texto, etiqueta):
    """
    Extrae contenido encerrado dentro de etiquetas HTML/XML específicas en un texto.

    Args:
        texto (str): El texto fuente que contiene las etiquetas.
        etiqueta (str): La etiqueta de la que se quiere extraer el contenido.

    Returns:
        list: Lista con el contenido encontrado entre las etiquetas.
    """
    patron = fr"<{etiqueta}>(.*?)</{etiqueta}>"
    return re.findall(patron, texto, re.DOTALL)

# Función para procesar los diálogos de dos oradores
def procesar_dialogos(dialogo):
    """
    Procesa un bloque de texto para separar los diálogos de Orador 1 y Orador 2.

    Args:
        dialogo (str): Texto fuente que contiene los diálogos.

    Returns:
        tuple: Dos listas con los diálogos de Orador 1 y Orador 2, respectivamente.
    """
    # Patrones para extraer diálogos de Orador 1 y Orador 2
    orador1_patron = r"Orador 1: (.*?)(?=(Orador 2:|$))"
    orador2_patron = r"Orador 2: (.*?)(?=(Orador 1:|$))"

    # Encontrar diálogos y limpiar espacios
    orador1_dialogos = [d[0].strip() for d in re.findall(orador1_patron, dialogo, re.DOTALL)]
    orador2_dialogos = [d[0].strip() for d in re.findall(orador2_patron, dialogo, re.DOTALL)]

    return orador1_dialogos, orador2_dialogos

## Input

In [61]:
# Librería
import uuid
import os, io, subprocess, ffmpeg
from groq import Groq

from pydantic import BaseModel, ConfigDict, Field

In [62]:
class Audio(BaseModel):
    audio_bytes: io.BytesIO
    id: str = Field(default=str(uuid.uuid4()))
    audio_path: str = Field(default=None)
    input_format: str = Field(default="wav")
    output_format: str = Field(default="ogg")

    def __init__(self, **data):
        super().__init__(**data)
        self.convert_to_ogg()
        self.audio_path=self.id+"."+self.output_format

    class Config:
        arbitrary_types_allowed = True

    def convert_to_ogg(self):
        """
        Generar el audio temporal en formato ogg
        """
        output_file=self.id+"."+self.output_format
        self.audio_bytes.seek(0)
        bytesio_to_ogg(self.audio_bytes, self.input_format, output_file)

class STT(BaseModel):
  client: Groq = None

  class Config:
        arbitrary_types_allowed = True

  def __init__(self, **data):
        super().__init__(**data)
        self.client = Groq(api_key=GROQ_API_KEY)

  def obtener_texto(self, audio: Audio):
        """
        Transcribes audio and returns the text.
        """

        # Check if the file exists
        if not os.path.exists(audio.audio_path):
            return "Error: The audio file could not be found. Please try again."

        try:
            # Open the OGG file and send it for transcription
            with open(audio.audio_path, "rb") as audio_file:
                transcription = self.client.audio.transcriptions.create(
                    file=audio_file,
                    model="whisper-large-v3-turbo",
                    language="es",
                    temperature=0.0
                )
                return transcription.text
        except Exception:
            # Return a generic error message for Streamlit
            return "An error occurred while processing the audio. Please try again."

In [ ]:
stt = STT()

# test

audio_data = io.BytesIO()
with open("temp.wav", "rb") as f:
    audio_data.write(f.read())
audio_data.seek(0)

audio1 = Audio(audio_bytes=audio_data)

result = stt.obtener_texto(audio1)

print(result)

Converted to OGG: 7b3d6d91-aee0-458f-a3be-49dfe8f72855.ogg
 Hola, ¿qué tal? Hola, ¿qué tal?


## Generation Talk

In [87]:
import re

class Talker():
  prompt: str = generation_prompt
  model: str = "Meta-Llama-3.1-70B-Instruct"
  client: OpenAI = None

  class Config:
        arbitrary_types_allowed = True

  def __init__(self, **data):
        super().__init__(**data)
        self.client = OpenAI(
            base_url="https://api.sambanova.ai/v1/",
            api_key=SAMBANOVA_API_KEY,
        )

  def response(self, query):

    completion = self.client.chat.completions.create(
        model=self.model,
        messages=[
            {
                "role": "user",
                "content": self.prompt.format(topic=query),
            }
        ],
        stream=False,
    )

    response = completion.choices[0].message.content

    # Procesar el texto
    resultado = extraer_contenido(response, "resultado")[0].strip()
    dialogo = extraer_contenido(response, "dialogo")[0].strip()

    # Procesar diálogos
    orador1_dialogos, orador2_dialogos = procesar_dialogos(dialogo)

    return resultado, orador1_dialogos, orador2_dialogos, response

In [106]:
print(response)

<planificacion_podcast>

**Estructura del Podcast**

**Tema:** Cáncer de mama en perras

**Subtemas:**

1.  **Causas y factores de riesgo**
    *   Pregunta clave del Orador 2: ¿Cuáles son los principales factores de riesgo para que una perra desarrolle cáncer de mama?
    *   Punto principal del Orador 1: Explicar cómo la genética, la edad y la condición reproductiva (como la esterilización) influyen en el riesgo de cáncer de mama en perras.
    *   Anécdota o analogía: Comparar la importancia de la esterilización en perras con la práctica de la mastectomía preventiva en humanos para reducir el riesgo de cáncer de mama.
    *   Tangentes interesantes o preguntas inusuales del Orador 2: ¿Cómo se compara el riesgo de cáncer de mama en perras con el de otros animales domésticos? ¿Hay alguna relación entre la alimentación y el riesgo de cáncer de mama en perras?
2.  **Síntomas y diagnóstico**
    *   Pregunta clave del Orador 2: ¿Cuáles son los síntomas más comunes del cáncer de mama en p

In [103]:
# test

talker = Talker()

resultado, orador1_dialogos, orador2_dialogos, response = talker.response("cancer de mama en perras")

## Output

In [77]:
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import IPython

In [99]:
class TTS(BaseModel):
  client: gTTS = Field(default=gTTS)

  class Config:
        arbitrary_types_allowed = True

  def generar_audio_text(self, text):
        """
        Reads text
        """
        tts = self.client(text, lang='es', tld='es')

        fp = io.BytesIO()
        tts.write_to_fp(fp)
        # fp.seek(0)

        return Audio(audio_bytes=fp, input_format="mp3")

  def generar_audio(self, texto, tld='com.mx', velocidad=1.0):

        """Genera un fragmento de audio a partir de texto."""
        tts = gTTS(text=texto, lang="es", tld=tld, slow=False)
        tts_file = "temp.mp3"
        tts.save(tts_file)
        audio = AudioSegment.from_file(tts_file)
        # Ajustar velocidad del audio
        return audio.speedup(playback_speed=velocidad)

  def generar_conversacion(self, id=str(uuid.uuid4()), orador1_dialogos=[], orador2_dialogos=[], velocidad=1.0, pausa_ms=500, idioma='es'):
    """
    Genera un audio combinado de una conversación entre dos oradores.

    Args:
        orador1_dialogos (list): Lista de diálogos de Orador 1.
        orador2_dialogos (list): Lista de diálogos de Orador 2.
        velocidad (float): Velocidad del habla (1.0 es la velocidad normal, >1.0 más rápido, <1.0 más lento).
        pausa_ms (int): Tiempo de pausa entre los diálogos en milisegundos.
        idioma (str): Idioma para gTTS (por defecto 'es' para español).

    Returns:
        None: Genera un archivo de audio llamado 'conversacion_completa.mp3'.
    """

    if len(orador1_dialogos)>1 and len(orador2_dialogos)>1:

      # Crear audio final fusionado
      audio_final = AudioSegment.silent(duration=0)  # Inicializar audio vacío

      # Alternar entre los diálogos de ambos oradores
      for o1, o2 in zip(orador1_dialogos, orador2_dialogos):
          audio_final += self.generar_audio(o1, 'us', velocidad)
          audio_final += AudioSegment.silent(duration=pausa_ms)  # Pausa entre oradores
          audio_final += self.generar_audio(o2, 'es', velocidad)
          audio_final += AudioSegment.silent(duration=pausa_ms)  # Pausa entre oradores

      # Si Orador 1 tiene más diálogos, añadir los restantes
      for i in range(len(orador2_dialogos), len(orador1_dialogos)):
          audio_final += self.generar_audio(orador1_dialogos[i])
          audio_final += AudioSegment.silent(duration=pausa_ms)  # Pausa entre oradores

      # Guardar el audio combinado
      audio_final.export(id+".mp3", format="mp3")

      return id+".mp3"


In [104]:
# Test
tts = TTS()

tts.generar_conversacion(orador1_dialogos=orador1_dialogos, orador2_dialogos=orador2_dialogos, velocidad=1.2)

ok


'267ca8d6-9c96-4006-808b-480e4b82c894.mp3'